In [98]:
import tensorflow as tf
import collections
from collections import namedtuple
import pickle
import parameters
import numpy as np

In [99]:
User2Vec = namedtuple('User2Vec', ['user_id', 'sent_ids', 'neg_ids', 'optimizer', 'loss', 'normalized_U'])

In [100]:
def hinge_loss(user_embeds, word_embeds, neg_sample_ids):
    pos_score = tf.matmul(user_embeds, word_embeds, transpose_a = True)
    print('pos_score: ', pos_score)
    
    neg_score = tf.tensordot(user_embeds, neg_sample_ids, [0,0])
    print('neg_score: ', neg_score)

    loss = tf.maximum(0.0, 1 - pos_score + neg_score)
    
    return loss

In [101]:
def build_model(sess, graph, embed_matrix_rows, n_users, embed_matrix):

    with graph.as_default():
    # Ops and variables pinned to the CPU because of missing GPU implementation
        with tf.device('/cpu:0'):
            global_step = tf.Variable(0, trainable=False)
            
            # u_j
            user_id = tf.placeholder(tf.int32, shape=())
            print('user_ids: ', user_id)
            U = tf.Variable(tf.random_uniform([n_users, embed_matrix_rows], -1.0, 1.0))
            print('U: ', U)
#             user_embed = tf.nn.embedding_lookup(U, user_id)
            user_embed = tf.slice(U, [0, user_id], [U.get_shape()[0], 1])
            print('user_embed: ', user_embed)

            # e_i
            E = tf.Variable(embed_matrix, dtype=tf.float32)
            print('E: ', E)
            sent_ids = tf.placeholder(tf.int32, shape=None)
            print('sent_ids: ', sent_ids)
            word_embeds = tf.nn.embedding_lookup(E, sent_ids)
            print('word_embeds :', word_embeds)
            
            # e_l
            neg_ids = tf.placeholder(tf.int32, shape=None)
            neg_sample_ids = tf.nn.embedding_lookup(E, neg_ids)

            loss = tf.reduce_mean(hinge_loss(user_embed, word_embeds, neg_sample_ids))
            
        # Construct the SGD optimizer using a learning rate of 1.0.
        optimizer = tf.train.GradientDescentOptimizer(.05).minimize(loss, global_step=global_step)

        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(U), 1, keep_dims=True))
        normalized_U = U / norm
        
        tf.global_variables_initializer().run()
        print("Initialized")

    model = User2Vec(user_id, sent_ids, neg_ids, optimizer, loss, normalized_U)

    return model

In [102]:
def train(sess, model, n_users):
    
    user_ids = np.arange(n_users)
    max_num_steps = 10
    
    user_idx = {}
    for prev_user, train, test, neg_samples in user_train_data:
        
        try:
            user_id = user_idx[prev_user]
        except KeyError:
            user_idx[prev_user] = len(user_idx)
    
        average_loss_step = max(parameters.checkpoint_step/10, 100)
    
        average_loss = 0
        for step in range(max_num_steps):
            
            for id in np.random.permutation(len(train)):
#                 print('train: ', train)
#                 print('train[id]', len(train[id]))
                
#                 print('train[id]', len(neg_samples[id]))
                print('neg samples: ', neg_samples)
                print('train: ', train)
                print('id: ', id)
                
                feed_dict = {model.user_id.name: user_idx[prev_user], model.sent_ids.name: train[id], model.neg_ids.name: neg_samples[id]}


            _, loss_val = sess.run([model.optimizer, model.loss], feed_dict=feed_dict)
            average_loss += loss_val
            
            if step % average_loss_step == 0:
                if step > 0:
                    average_loss /= average_loss_step
                print('Average loss at step: ', step, ': ', average_loss)
                average_loss = 0
                
    final_embeddings = model.normalized_U.eval()
    
    return final_embeddings

In [103]:
if __name__ == '__main__':
    
    # load pickled word embeddings
    # because we want the number of users which we pickled here
    embed_matrix, unigram_prob, wrd2idx, word_counter, n_users = pickle.load(open(parameters.output_pkl, 'rb'))
    print('embed matrix shape 0: ', embed_matrix.shape[0])
    print('embed matrix shape: ', embed_matrix.shape)
    print('embed matrix shape 1: ', embed_matrix.shape[1])
    print('embed matrix len: ', len(embed_matrix))
    
    user_train_data = pickle.load(open(parameters.output, 'rb'))

    graph = tf.Graph()
    with tf.Session(graph=graph) as sess:

        model = build_model(sess, graph, embed_matrix.shape[0], n_users, embed_matrix)
        
        user_embeddings = train(sess, model, n_users)
    

embed matrix shape 0:  78
embed matrix shape:  (78, 74)
embed matrix shape 1:  74
embed matrix len:  78
user_ids:  Tensor("Placeholder:0", shape=(), dtype=int32, device=/device:CPU:0)
U:  <tf.Variable 'Variable_1:0' shape=(86, 78) dtype=float32_ref>
user_embed:  Tensor("Slice:0", shape=(86, 1), dtype=float32, device=/device:CPU:0)
E:  <tf.Variable 'Variable_2:0' shape=(78, 74) dtype=float32_ref>
sent_ids:  Tensor("Placeholder_1:0", dtype=int32, device=/device:CPU:0)
word_embeds : Tensor("embedding_lookup:0", dtype=float32, device=/device:CPU:0)
pos_score:  Tensor("MatMul:0", shape=(1, ?), dtype=float32, device=/device:CPU:0)
neg_score:  Tensor("Tensordot:0", dtype=float32, device=/device:CPU:0)
Initialized
neg samples:  [[59, 20, 15, 21, 69, 3, 30, 30, 7, 65]]
train:  [[]]
id:  0


InvalidArgumentError: Matrix size-incompatible: In[0]: [86,1], In[1]: [0,74]
	 [[Node: MatMul = MatMul[T=DT_FLOAT, transpose_a=true, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Slice, embedding_lookup)]]

Caused by op 'MatMul', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 112, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.5/asyncio/base_events.p